In [399]:
from nltk.corpus import stopwords
import codecs
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold
from collections import defaultdict
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.svm import SVC
import time
from sklearn.metrics import f1_score

In [379]:
def read_it(item):
    f=codecs.open(item ,'r')
    f=f.read()
    f=f.replace("\n", " ")
    sent_tokens = sent_tokenize(f)
    tokens = [sent for sent in map(word_tokenize, sent_tokens)]
    list(enumerate(tokens))
    others = '“,”,’,—'
    stopwords_ = set(stopwords.words('english'))
    tokens_lower = [[word.lower() for word in sent] for sent in tokens]
    punctuation_ = set(string.punctuation)
    clean_token= [[word.replace(str(punctuation_),'') for word in sent] for sent in tokens_lower]

    def filter_tokens(sent):
        return([w for w in sent if not w in stopwords_ and not w in punctuation_ and not w in others])

    tokens_filtered = list(map(filter_tokens, clean_token))



    return tokens_filtered

In [380]:
def chunk(x, y = 30):
    l=len(x)
    v=l//y
    remainder=y
    all_series = []
    for loops in range(v+1):
        new_elem = []
        if loops > (v - 1):
            remainder = l%y
        if remainder > 0:
            for i in range(remainder):
                temp = (y * (loops)) + i
                new_elem.extend(x[temp])
            all_series.append(new_elem)
    return np.array(all_series,dtype=object)

In [381]:
def make_it_df(item,title,author):
    df = pd.DataFrame()
    df['txt']= chunk(read_it(item))
    df['title']= title
    df['author'] = author
    return df

In [382]:
bell = make_it_df("/Users/andrewargaez/Author_Classifier/WBT.txt",'For Whom the Bell Tolls','Ernest Hemmingway')
fta = make_it_df("/Users/andrewargaez/Author_Classifier/FTA.txt", "A Farewell to Arms", "Ernest Hemingway")

kar = make_it_df("/Users/andrewargaez/Author_Classifier/karmaz.txt", "The Brothers Karmazov", "Fyodor Dostoevsky")
cp = make_it_df("/Users/andrewargaez/Author_Classifier/CP.txt", "Crime and Punishment", "Fyodor Dostoevsky")

# gg = make_it_df("/Users/andrewargaez/Author_Classifier/GG.txt", "The Great Gatsby", "F. Scott Fitzgerald")
# sop = make_it_df("/Users/andrewargaez/Author_Classifier/sop.txt", "This Side of Paradise", "F. Scott Fitzgerald")

# pp = make_it_df("/Users/andrewargaez/Author_Classifier/PP.txt", "Pride and Prejudice", "Jane Austen")
# em = make_it_df("/Users/andrewargaez/Author_Classifier/emma.txt", "Emma", "Jane Austen")

# al= make_it_df("/Users/andrewargaez/Author_Classifier/alice.txt", "Alice in Woderland", "Lewis Carrol")
# tlg= make_it_df("/Users/andrewargaez/Author_Classifier/tlg.txt", "Through the Looking Glass", "Lewis Carrol")

# drac= make_it_df("/Users/andrewargaez/Author_Classifier/drac.txt", "Dracula", "Bram Stoker")
# lf= make_it_df("/Users/andrewargaez/Author_Classifier/LF.txt", "Lord of the Flies", "William Golding")
# pet = make_it_df("/Users/andrewargaez/Author_Classifier/Peter.txt", "Peter Pan", "J. M. Barrie")

# pdg = make_it_df("/Users/andrewargaez/Author_Classifier/pdg.txt", "Picture of Dorian Gray", "Oscar Wilde")

# ti  = make_it_df("/Users/andrewargaez/Author_Classifier/TI.txt", "Treasure Island", "Robert Louis Stevenson")

# huck = make_it_df("/Users/andrewargaez/Author_Classifier/huck.txt", "Adventures of Huckleberry Fin", "Mark Twain")
# ts = make_it_df("/Users/andrewargaez/Author_Classifier/ts.txt", "Adventures of Tom Sawyer", "Mark Twain")

# wow = make_it_df("/Users/andrewargaez/Author_Classifier/wow.txt", "The War of the Worlds", "H. G. Wells")
# im = make_it_df("/Users/andrewargaez/Author_Classifier/IM.txt", "The Invisible Man", "H. G. Wells")



In [383]:
new_df =pd.concat([bell, kar],ignore_index=True)
test= pd.concat([fta, cp],ignore_index=True)


,txt,title,author
0,"[lay, flat, brown, pine-needled, floor, forest...",For Whom the Bell Tolls,Ernest Hemmingway
1,"[hungry, yes, young, man, said, eat, later, ca...",For Whom the Bell Tolls,Ernest Hemmingway
2,"[young, man, whose, name, robert, jordan, extr...",For Whom the Bell Tolls,Ernest Hemmingway
3,"[never, attacks, golz, said, make, mine, artil...",For Whom the Bell Tolls,Ernest Hemmingway
4,"[see, people, mountains, get, many, men, need,...",For Whom the Bell Tolls,Ernest Hemmingway
...,...,...,...
1038,"[want, buried, churchyard, snegiryov, wailed, ...",The Brothers Karmazov,Fyodor Dostoevsky
1039,"[one, moment, went, coffin, set, straight, cov...",The Brothers Karmazov,Fyodor Dostoevsky
1040,"[half‐way, snegiryov, suddenly, stopped, stood...",The Brothers Karmazov,Fyodor Dostoevsky
1041,"[serious, earnest, expression, looked, one, an...",The Brothers Karmazov,Fyodor Dostoevsky


In [384]:
def squeaky_clean(new_df):
    others = '“,”,’,—,_,.,——,--'
    vals = list(new_df['txt'].values)
    arr=[]

    for _sent in vals:
        sent=[]
        for word in _sent:
            for char in others:
                word = word.replace(char,'')
            sent.append(word)      
        arr.append(sent)
    return pd.Series(arr)

In [385]:
new_df['txt']= squeaky_clean(new_df)

new_df

,txt,title,author
0,"[lay, flat, brown, pineneedled, floor, forest,...",For Whom the Bell Tolls,Ernest Hemmingway
1,"[hungry, yes, young, man, said, eat, later, ca...",For Whom the Bell Tolls,Ernest Hemmingway
2,"[young, man, whose, name, robert, jordan, extr...",For Whom the Bell Tolls,Ernest Hemmingway
3,"[never, attacks, golz, said, make, mine, artil...",For Whom the Bell Tolls,Ernest Hemmingway
4,"[see, people, mountains, get, many, men, need,...",For Whom the Bell Tolls,Ernest Hemmingway
...,...,...,...
1038,"[want, buried, churchyard, snegiryov, wailed, ...",The Brothers Karmazov,Fyodor Dostoevsky
1039,"[one, moment, went, coffin, set, straight, cov...",The Brothers Karmazov,Fyodor Dostoevsky
1040,"[half‐way, snegiryov, suddenly, stopped, stood...",The Brothers Karmazov,Fyodor Dostoevsky
1041,"[serious, earnest, expression, looked, one, an...",The Brothers Karmazov,Fyodor Dostoevsky


In [386]:
test['txt']= squeaky_clean(test)

In [387]:
def get_data(item):
    data=[]
    for row in item.txt:
        data.append(' '.join(row))
    labels = item.author
    le = LabelEncoder()
    y = le.fit_transform(labels)
    return data, np.array(y)

In [388]:
def vectorizer(data):
    tfidf = TfidfVectorizer()
    X = tfidf.fit_transform(data).toarray()
    return X,tfidf

In [389]:
data,y =get_data(new_df)


In [364]:
data_train, data_test, y_train, y_test = train_test_split(data, y, test_size=.5)

In [375]:
X_train, tf = vectorizer(data_train)

In [403]:
test_data, test_y =get_data(test)

In [405]:
test_tfidf = tf.transform(test_data)

In [394]:
X_test=tf.transform(data_test)
X_test.shape

(522, 12053)

In [395]:
model  = RandomForestClassifier()
#start = time.time()
model.fit(X_train, y_train)

RandomForestClassifier()

In [396]:
y_hat = model.predict(X_test)
y_test.shape

(522,)

In [406]:
test_hat= model.predict(test_tfidf)

In [400]:
f1_score(y_test, np.ones(len(y_test)))
score

0.5182341650671785

In [402]:
f1_score(y_test, y_hat)

0.9984544049459042

In [ ]:
f1_score(_test, test_hat)

In [193]:
model  = RandomForestClassifier()
#start = time.time()
model.fit(X_train, y_train)
#y_hat = model.predict(X_test.T)
#trained = time.time()
score = model.score(X_test,y_test)
#tested = time.time()

ValueError: Found input variables with inconsistent numbers of samples: [261, 782]

In [ ]:
def tune_naive_bayes(data, y):
    print("tuning naive bayes...")
    kfold = KFold(5)
    alphas = np.concatenate((np.arange(0, 0.1, 0.02), np.arange(.1, 1.3, 0.1)))
    scores = defaultdict(list)
    for train_index, test_index in kfold.split(data):
        data_train, data_test = data[train_index], data[test_index]
        y_train, y_test = y[train_index], y[test_index]
        tfidf = TfidfVectorizer()
        X_train = tfidf.fit_transform(data_train)
        print(X_train.shape)
        X_test = tfidf.transform(data_test)
        for alpha in alphas:
            nb = MultinomialNB(alpha=alpha)
            nb.fit(X_train, y_train)
            scores[alpha].append(nb.score(X_test, y_test))

    print("alpha  score")
    for alpha in alphas:
        print(" %.2f  %f" % (alpha, np.average(scores[alpha])))

In [ ]:
def run_models(data, y,t,t2):
    data_train, data_test, y_train, y_test = train_test_split(data, y)
    

    tfidf = TfidfVectorizer()
    X_train = tfidf.fit_transform(data_train).toarray()
    X_test = tfidf.transform(data_test).toarray()




    print("running models...")
    models = [("Random Forest", RandomForestClassifier()),
              ("Decision Tree", DecisionTreeClassifier()),
              ("kNN", KNeighborsClassifier()),  
              ("Naive Bayes", MultinomialNB()),
              #("SVM", OneVsRestClassifier(SVC())),
              ("Logistic", OneVsRestClassifier(LogisticRegression()))]

    print("%20s %7s %9s %9s" % ("Name", "Score", "TrainTime", "TestTime"))

    for name, model in models:
        start = time.time()
        model.fit(X_train, y_train)
        y_hat = model.predict(X_test.T)
        trained = time.time()
        score = model.score(y_hat,y_test)
        tested = time.time()

        # Silly stuff to make it print nicely
        print("%20s   %.3f %9s %9s" % (name, score,
                                       str(round(trained - start, 2)),
                                       str(round(tested - trained, 2))))

In [ ]:
t, t2 = get_data(new_df)
t = vectorizer(t)
len(t)

1043

In [ ]:
test1, test2 = get_data(test)
print(sum(test2))
tune_naive_bayes(test1, test2)
run_models(test1,test2,t,t2)

397
tuning naive bayes...
(507, 10681)
(507, 10709)


/Users/andrewargaez/opt/anaconda3/lib/python3.8/site-packages/sklearn/naive_bayes.py:508: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '
/Users/andrewargaez/opt/anaconda3/lib/python3.8/site-packages/sklearn/naive_bayes.py:508: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '


(507, 10410)
(507, 10495)


/Users/andrewargaez/opt/anaconda3/lib/python3.8/site-packages/sklearn/naive_bayes.py:508: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '
/Users/andrewargaez/opt/anaconda3/lib/python3.8/site-packages/sklearn/naive_bayes.py:508: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '


(508, 10723)
alpha  score
 0.00  0.911811
 0.02  0.988976
 0.04  0.990551
 0.06  0.990551
 0.08  0.985814
 0.10  0.984239
 0.20  0.935421
 0.30  0.892901
 0.40  0.847232
 0.50  0.803137
 0.60  0.749594
 0.70  0.713373
 0.80  0.688176
 0.90  0.675578
 1.00  0.661405
 1.10  0.645657
 1.20  0.642507
running models...
                Name   Score TrainTime  TestTime


/Users/andrewargaez/opt/anaconda3/lib/python3.8/site-packages/sklearn/naive_bayes.py:508: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '


ValueError: X has 159 features, but DecisionTreeClassifier is expecting 10437 features as input.

In [ ]:
#data, y = get_data(new_df)
#tune_naive_bayes(data, y)
#run_models(data, y,test1,test2)